# This is the testing notebook for bens decision tree adventure!

In [1]:
! pip install -r requirements.txt

## Import functions

In [2]:
import pandas as pd

from classification_trees.utility.split_data import split_data_on_float
from classification_trees.classification_tree import Classification_tree

## Load in test dataset

Below is a sample row of this small 200 row dataset. It is a dataset going into medical history of people, and what drugs they use to remedy their illnesses.

In [3]:
df = pd.read_csv('./data/drug200.csv')

In [4]:
df.head(3)

,Age,Sex,BP,Cholesterol,Na_to_K,Drug
0,23,F,HIGH,HIGH,25.355,drugY
1,47,M,LOW,HIGH,13.093,drugC
2,47,M,LOW,HIGH,10.114,drugC


### Make cols lower case

In [5]:
df.columns = [col.lower() for col in df.columns]

### Encode non numeric columns

In [6]:
def encode_column(column:str, column_value: str) -> int:
    keys = list(df[column].value_counts().keys())
    keys.sort()
    values = list(range(0,len(keys)))

    encoder = dict(zip(keys,values))

    return encoder[column_value]

In [7]:
columns_to_be_encoded = ["age","sex","bp","cholesterol","drug"]

In [8]:
for column in columns_to_be_encoded:
    df[column] = df[column].apply(lambda column_value: encode_column(column = column, column_value= column_value))

### Make columns binary

In [9]:
df = df[(df['drug'] == 1) | (df['drug'] == 0)]

## Splitting the dataset

This function will split the dataset up into two sets based on some columnar value.

In [10]:
d = split_data_on_float(df, 'age',20,'drug')

### Finding the best row to split the data on

We look through ALL non classification rows, then run split_data_on_float with the unique values from each row. We choose the row that provides the least entropy.

In [11]:
tree = Classification_tree(df)